In [1]:
edges = []
n =[]
#we import data that we need and we fill them 
with open("wiki-topcats-reduced.txt\wiki-topcats-reduced.txt") as mytxt:
    for line in mytxt:
        i,j= line.split()
        n.extend((int(i),int(j)))
        edges.append([int(i),int(j)])

In [2]:
#we eliminate duplicates from nodes
nodes = set(n)

In [3]:
#we create a dictionary with articoles for each category, we take only articles in the reduced version of the graph 
#and only categories with more than 3500 articoles
cat_dic = {}
with open ("wiki-topcats-categories.txt")as dc:
    for line in dc:
        l1 = line.replace("Category:","")
        l2 = l1.split(";")
        k = l2[0]
        art = list(map(int,l2[1].split()))
        for i in art :
            if i not in nodes:
                art.remove(i) 
        if len(art)>3500:
             cat_dic[k] = art

In [4]:
#we store the graph in a dictionary saving from each source node a list with end nodes of its edges
from collections import defaultdict
g = defaultdict(list)
for e in edges:
    g[e[0]].append(e[1])
    

In [5]:
#here we create the bfs function, in this version the function save distances from the starting node to a set of goal 
#nodes in a dictionary
from collections import deque
def bfs(graph, start, goal,dists):
    visited = set([start])
    queue = deque([(start, [])])
    goal1 = set(goal)
    while queue:
        current, path = queue.popleft()
        visited.add(current)
        for neighbor in graph[current]:
            if neighbor in goal1:
                dists[neighbor].append(len(path + [current, neighbor])-1)
                goal1.remove(neighbor)
            if neighbor in visited:
                continue
            queue.append((neighbor, path + [current]))
            visited.add(neighbor)   
      

In [6]:
#here we define the dictionary with distances
dists = defaultdict(list)

In [7]:
#we create a list with all nodes from all categories except the input category
int_nodes=[]
for k in cat_dic.keys():
    if k != 'Association_football_goalkeepers':
            l1 = cat_dic[k]
            l1 = [a for a in l1 if a not in cat_dic['Association_football_goalkeepers']]
    int_nodes.extend(l1)

In [ ]:
#we execute the bfs for each node of the input category, so we'll have in dists for each node a list with all lengths
#of shortest path  to that node having as starting node a node from the input category
for a in cat_dic['Association_football_goalkeepers']:
    bfs(g,a,int_nodes,dists)


In [32]:
#Here we check if in some cases there was no link between one node of the input categories and one of the other 
#categories, we simply add 200000 to the list of distances in that case. With this procedure we don't keep track of what
#node wasn't connected to the target node, but since our goal is the reasearch of the median this information is not 
#important
for a in dists:
    if len(dists[a])<len("Association_football_goalkeepers"):
        l1=[200000]*(len(cat_dic['Association_football_goalkeepers']-len(dists[a]))
        dists[a].extend(l1)

In [33]:
import statistics as st
medians = []
for k in cat_dic.keys():
    if k != 'Association_football_goalkeepers':
            d = []
            for a in cat_dic[k]:
                  d.extend(dists[a])
            med = st.median(d)
            medians.append([k,med])

In [34]:
medians

[['English_footballers', 7.0],
 ['The_Football_League_players', 7],
 ['Association_football_forwards', 7],
 ['Association_football_goalkeepers', 7.0],
 ['Association_football_midfielders', 7.0],
 ['Association_football_defenders', 7],
 ['Living_people', 6.0],
 ['Harvard_University_alumni', 5],
 ['Major_League_Baseball_pitchers', 6.0],
 ['Members_of_the_United_Kingdom_Parliament_for_English_constituencies', 6],
 ['Indian_films', 6.0],
 ['Year_of_death_missing', 200000.0],
 ['Year_of_birth_missing_(living_people)', 6.0],
 ['Rivers_of_Romania', 7.0],
 ['Main_Belt_asteroids', 200000.0],
 ['Asteroids_named_for_people', 11],
 ['English-language_albums', 5],
 ['British_films', 5],
 ['English-language_films', 4],
 ['American_films', 4],
 ['People_from_New_York_City', 5.0],
 ['American_television_actors', 200000.0],
 ['Debut_albums', 6.0],
 ['Black-and-white_films', 5.0],
 ['Year_of_birth_missing', 13],
 ['Place_of_birth_missing_(living_people)', 6],
 ['American_military_personnel_of_World_War_

In [35]:
#we insert in position 0 of the ranking block the input category
medians.insert(0,['Association_football_goalkeepers',0])

In [36]:
#we sort the block according to the median
medians.sort(key = lambda x : x[1])

In [37]:
medians

[['American_film_actors', 0],
 ['English-language_films', 4],
 ['American_films', 4],
 ['Harvard_University_alumni', 5],
 ['English-language_albums', 5],
 ['British_films', 5],
 ['People_from_New_York_City', 5.0],
 ['Black-and-white_films', 5.0],
 ['American_military_personnel_of_World_War_II', 5],
 ['Living_people', 6.0],
 ['Major_League_Baseball_pitchers', 6.0],
 ['Members_of_the_United_Kingdom_Parliament_for_English_constituencies', 6],
 ['Indian_films', 6.0],
 ['Year_of_birth_missing_(living_people)', 6.0],
 ['Debut_albums', 6.0],
 ['Place_of_birth_missing_(living_people)', 6],
 ['Windows_games', 6],
 ['English_footballers', 7.0],
 ['The_Football_League_players', 7],
 ['Association_football_forwards', 7],
 ['Association_football_goalkeepers', 7.0],
 ['Association_football_midfielders', 7.0],
 ['Association_football_defenders', 7],
 ['Rivers_of_Romania', 7.0],
 ['Asteroids_named_for_people', 11],
 ['Year_of_birth_missing', 13],
 ['Year_of_death_missing', 200000.0],
 ['Main_Belt_aste

In [ ]:
#saving medians with pickle
import pickle
with open('medians', 'wb') as f:
        pickle.dump(medians, f)

In [ ]:
#saving dists with pickle
with open('dists.pickle', 'wb') as f:
    pickle.dump(dists, f, protocol=pickle.HIGHEST_PROTOCOL)

## RQ2 part 2

In [21]:
end_n =  defaultdict(list)
for e in edges:
    end_n[e[1]].append(e[0])

In [22]:
dic_score = {}
for a in cat_dic['American_film_actors']:
      dic_score[a]=0
for a in int_nodes:
      dic_score[a]=0

In [41]:
for a in cat_dic['American_film_actors']:
    summa = 0
    for ar in end_n[a]:
        if ar in cat_dic["American_film_actors"]:
            summa = summa + 1
    dic_score[a]= summa

In [42]:
visited = list(cat_dic['American_film_actors'])
for i in range(1,29):
    for a in cat_dic[medians[i][0]]:
            summa = 0
            for ar in end_n[a]:
                    if ar in visited:
                        summa = summa + dic_score[ar]  
                    elif ar in cat_dic[medians[i][0]]:
                        summa = summa +1
            dic_score[a]=summa  
    visited.extend(cat_dic[medians[i][0]])

KeyboardInterrupt: 

In [ ]:
ord_art = []
for i in range(29):
    l = []
    for a in cat_dic[medians[i][0]]:
          l.append([a,dic_score[a]])
    l = l.sort(key = lambda x : x[1])
    ord_art.append([medians[i][0],l])

In [28]:
for a in cat_dic["English-language_films"]:
    summa = 0
    for ar in end_n[a]:
        if ar in cat_dic['American_film_actors']:
            summa = summa + dic_score[ar]
        if ar in cat_dic["English-language_films"]:
            summa = summa +1
    dic_score[a]= summa

In [ ]:
for i in range(1,29):
    for ar in end_n[a]:
            if ar in visited:
                 summa = summa + dic_score[ar]  

In [43]:
import networkx as nx
edges = []
n =[]
#we import data that we need and we fill them 
with open("wiki-topcats-reduced.txt\wiki-topcats-reduced.txt") as mytxt:
    for line in mytxt:
        i,j= line.split()
        n.extend((int(i),int(j)))
        edges.append((int(i),int(j)))
nodes = set(n)
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

In [ ]:
for i in range(29):
    l = []
    l = l.extend(cat_dic[medians[i][0]])
    s = G.subgraph(l)
    for n in s:
        
        